In [51]:
import pandas as pd
import os
from src.Get_data_DB import DataTransformer

transformer=DataTransformer()

# Lấy số mess theo thời gian, khoá học trên ME
Query_FA= """ SELECT DATE(DATE_ADD(lp.created_at, INTERVAL 7 HOUR)) AS Date, 
                      p.code AS Ten_khoa_hoc,
                      p.id AS Ma_khoa_hoc,
                      COUNT(*) AS So_FA
                FROM leads l
                JOIN leads_products lp ON lp.lead_id = l.id
                JOIN products p ON lp.product_id = p.id
                WHERE l.utm_source = 'FA'
                  AND DATE(DATE_ADD(lp.created_at, INTERVAL 7 HOUR)) >= '2024-01-01'
                GROUP BY Date, Ten_khoa_hoc, Ma_khoa_hoc;
                """
Query_KH=""" select Ma_khoa_hoc, Ten_khoa_hoc 
            from Dim_Khoa_hoc
"""
df_count_FA=transformer.fetch_from_mysql(Query_FA)
df_KH=transformer.fetch_from_sql_server(Query_KH)
df_cf1=pd.read_csv("~/DWH_Cole_Project/data_tmp/spend_C9_PAUSED.csv")
df_cf2=pd.read_csv("~/DWH_Cole_Project/data_tmp/spend_Cole8_PAUSED.csv")

In [42]:
df_cf1['Ten_khoa_hoc'] = df_cf1['Campaign Name'].str.split('_').str[1]

df_cf2['Ten_khoa_hoc'] = df_cf2['Campaign Name'].str.split('_').str[1]

In [43]:
df_cf = pd.concat([df_cf1, df_cf2], ignore_index=True)

In [21]:
df_count_FA

,Date,Ten_khoa_hoc,Ma_khoa_hoc,So_FA
0,2024-01-01,ML.I,529,7
1,2024-01-01,NCLC,528,11
2,2024-01-02,ML.I,529,3
3,2024-01-02,NCLC,528,20
4,2024-01-03,DA.COMBO02,516,2
...,...,...,...,...
1681,2025-07-07,DE.COMBO01,515,3
1682,2025-07-07,ML.I,529,5
1683,2025-07-07,PSE2025,554,2
1684,2025-07-08,AI.2025,553,2


In [44]:
df_cf

,Campaign ID,Campaign Name,Date,Spend,Ten_khoa_hoc
0,120203877065080236,FA_NCLC_LAD_CVS_page DTNH -1/2,2024-01-02,685126.0,NCLC
1,120203877065080236,FA_NCLC_LAD_CVS_page DTNH -1/2,2024-01-03,492904.0,NCLC
2,120203877065080236,FA_NCLC_LAD_CVS_page DTNH -1/2,2024-01-04,355961.0,NCLC
3,120203877065080236,FA_NCLC_LAD_CVS_page DTNH -1/2,2024-01-05,234781.0,NCLC
4,120203877065080236,FA_NCLC_LAD_CVS_page DTNH -1/2,2024-01-06,216659.0,NCLC
...,...,...,...,...,...
3605,120230019048670679,FA_ML.I_BinhND_CVS_Page BOT_13_06,2025-06-22,277262.0,ML.I
3606,120230019048670679,FA_ML.I_BinhND_CVS_Page BOT_13_06,2025-06-23,267474.0,ML.I
3607,120230019048670679,FA_ML.I_BinhND_CVS_Page BOT_13_06,2025-06-24,255531.0,ML.I
3608,120230019048670679,FA_ML.I_BinhND_CVS_Page BOT_13_06,2025-06-25,223639.0,ML.I


In [45]:
df_cf['Ten_khoa_hoc'] = df_cf['Ten_khoa_hoc'].str.upper()

# Thay thế giá trị 'BI' bằng 'BI.01'
df_cf['Ten_khoa_hoc'] = df_cf['Ten_khoa_hoc'].replace('BI', 'BI.01', regex=False)

In [46]:
df_count_FA["Date"] = pd.to_datetime(df_count_FA["Date"]).dt.date
df_cf["Date"] = pd.to_datetime(df_cf["Date"]).dt.date


df_FA = df_cf.merge(df_count_FA, on=['Ten_khoa_hoc', 'Date'], how='left')
df_FA=df_FA[['Spend','So_FA','Date','Ten_khoa_hoc']]
df_FA['So_FA'] = df_FA['So_FA'].fillna(0)
df_FA = df_FA.rename(columns={
    'Spend': 'Chi_phi',
    'Date': 'Thoi_gian'
})
df_FA=df_FA.merge(df_KH,on='Ten_khoa_hoc',how='inner')
df_FA=df_FA[['Chi_phi','So_FA','Thoi_gian','Ma_khoa_hoc']]
df_FA = df_FA.groupby(['Thoi_gian', 'Ma_khoa_hoc'])[['Chi_phi', 'So_FA']].sum().reset_index()


In [47]:
df_L1_mess_bonus=df_cf.merge(df_count_FA, on=['Ten_khoa_hoc', 'Date'], how='right')


In [50]:
df_L1_mess_bonus=df_L1_mess_bonus[['Date','Spend','Ma_khoa_hoc','So_FA']]
df_L1_mess_bonus = df_L1_mess_bonus.rename(columns={
    'Spend': 'Chi_phi',
    'Date': 'Thoi_gian',
    'So_FA': 'So_L1_mess'
})
df_L1_mess_bonus = df_L1_mess_bonus[df_L1_mess_bonus['Chi_phi'].isna()]
df_L1_mess_bonus['Chi_phi'] = df_L1_mess_bonus['Chi_phi'].fillna(0)
df_L1_mess_bonus = df_L1_mess_bonus.groupby(['Thoi_gian', 'Ma_khoa_hoc'])[['Chi_phi', 'So_L1_mess']].sum().reset_index()

KeyError: "['Date', 'Spend', 'So_FA'] not in index"

In [49]:

df_FA.to_csv("/tmp/join1.csv")
df_L1_mess_bonus.to_csv("/tmp/join2.csv")

---

In [51]:
import os
import time
import requests
import pandas as pd
from datetime import date
from dotenv import load_dotenv
from src.Get_data_DB import DataTransformer

# Load biến môi trường từ file .env
load_dotenv()

transformer = DataTransformer()

ACCESS_TOKENS = {
    "C9": os.getenv("Cole_token"),
    "Cole8": os.getenv("BM_token")
}

paused_campaign_query = """
    SELECT STT AS campaign_id, 
           Chien_dich AS campaign_name,
           Ngay_bat_dau,
           Account
    FROM Chien_dich_Meta
    WHERE Account IN ('C9','Cole8')
      AND Trang_thai = 'PAUSED'
"""
df = transformer.fetch_from_sql_server(paused_campaign_query)

# Hàm gọi API an toàn (retry-safe)
def safe_request(url, params, max_retries=3, delay=240):
    for attempt in range(max_retries):
        try:
            res = requests.get(url, params=params if '?' not in url else None)
            res.raise_for_status()
            return res.json()
        except requests.exceptions.HTTPError as e:
            try:
                error_data = res.json().get("error", {})
                if error_data.get("code") == 17:
                    print(f"⏳ Đợi {delay}s vì giới hạn API... (thử lại {attempt+1}/{max_retries})")
                    time.sleep(delay)
                else:
                    raise e
            except Exception:
                raise e
    raise Exception("🚫 Vượt quá số lần retry khi gọi API.")

# Hàm lấy danh sách ad_id từ campaign_id
def fetch_ads_from_campaign(campaign_id, access_token):
    url = f"https://graph.facebook.com/v20.0/{campaign_id}/ads"
    params = {
        "access_token": access_token,
        "fields": "id,name",
        "limit": 100
    }

    ad_list = []
    while url:
        data = safe_request(url, params)
        ad_list.extend(data.get("data", []))
        url = data.get("paging", {}).get("next")
        time.sleep(1.5)  # ⏳ Nghỉ giữa các page

    return ad_list

# Xử lý từng tài khoản
today = date.today().strftime("%Y-%m-%d")

for account in ["C9", "Cole8"]:
    access_token = ACCESS_TOKENS[account]
    account_campaigns = df[df["Account"] == account].to_dict(orient="records")

    ad_rows = []

    print(f"\n🔍 Đang xử lý tài khoản {account} - Số chiến dịch: {len(account_campaigns)}")

    for campaign in account_campaigns:
        campaign_id = campaign["campaign_id"]
        campaign_name = campaign["campaign_name"]

        try:
            print(f"📥 Lấy ad_id từ campaign: {campaign_name} ({campaign_id})")
            ads = fetch_ads_from_campaign(campaign_id, access_token)

            for ad in ads:
                ad_rows.append({
                    "Campaign ID": campaign_id,
                    "Campaign Name": campaign_name,
                    "Ad ID": ad["id"],
                    "Ad Name": ad.get("name", "No name")
                })

            time.sleep(2.5)  # ⏳ Delay giữa các campaign
        except Exception as e:
            print(f"⚠️ Lỗi campaign {campaign_name} ({campaign_id}): {e}")

    # Ghi file kết quả
    df_ads = pd.DataFrame(ad_rows)
    output_path = os.path.expanduser(f"~/DWH_Cole_Project/data_tmp/ads_{account}_PAUSED.csv")
    df_ads.to_csv(output_path, index=False)
    print(f"✅ Đã ghi file: {output_path}")



🔍 Đang xử lý tài khoản C9 - Số chiến dịch: 224
📥 Lấy ad_id từ campaign: FA_NCLC_LAD_CVS_page DTNH -1/2 (120203877065080236)
📥 Lấy ad_id từ campaign: FA_ITBA_admin_CVS_ColeMoi_03_01_24 (120203898492820236)
📥 Lấy ad_id từ campaign: FA_DA.COMBO01_LVL_CVS_3_1 (120203912042170236)
📥 Lấy ad_id từ campaign: FA_ITBA_admin_Mess_ColeMoi_03_01_24 (120203914513830236)
📥 Lấy ad_id từ campaign: FA_ITBA_admin_CVS_ColeMoi_03_01_24 - Bản sao (120203918020850236)
📥 Lấy ad_id từ campaign: FA_NCLC_admin_CVS_page DTNH - 25_12 - Bản sao 2 (120203922130710236)
📥 Lấy ad_id từ campaign: FA_ITBA_admin_Mess_auto_ColeMoi_03_01_24 (120203925239800236)
📥 Lấy ad_id từ campaign: FA_AI.COMBO02_LAD_CVS_4/1 (120203930604270236)
📥 Lấy ad_id từ campaign: FA_ITBA_admin_CVS_ColeMoi_04_01_24 (120203950405090236)
📥 Lấy ad_id từ campaign: FA_NCLC_admin_CVS_page DTNH - 14_12 - Bản sao (120203976935940236)
📥 Lấy ad_id từ campaign: FA_ITBA_admin_Mess_KHTN_ColeMoi_06_01_24 (120203996651720236)
📥 Lấy ad_id từ campaign: FA_ITBA_adm